In [ ]:
""" Задачи """

Оценка разных статистик
  

Мы выяснили, что в эксперименте «Refactoring backend» в экспериментальной группе среднее время загрузки увеличилось, а 99% квантиль уменьшился. Проверьте значимость отличий других квантилей.

Данные эксперимента «Refactoring backend»: df_web_logs.csv и experiment_users.csv. Эксперимент проводился с 2022-04-05 по 2022-04-12. Измерения времени обработки запросов считаем независимыми. При проверке используйте нормальный доверительный интервал.

In [1]:
import os
from datetime import datetime
import numpy as np
import pandas as pd
from scipy import stats

In [2]:
def read_file(file_name):
    if not os.path.exists(file_name):
        raise FileNotFoundError(f'Файл {file_name} не найден')
    return pd.read_csv(file_name)

In [3]:
df_web_logs = read_file('df_web_logs.csv')
df_users = read_file('experiment_users.csv')

In [4]:
df_web_logs['date'] = pd.to_datetime(df_web_logs['date'])

start_date = datetime(2022, 4, 5)
end_date = datetime(2022, 4, 12)

df_metrics = df_web_logs[(df_web_logs['date']>=start_date) 
                         & (df_web_logs['date']<end_date)][['user_id', 'load_time']].reset_index(drop=True)

df = pd.merge(df_users, df_metrics, on='user_id', how='left')[['pilot', 'load_time']]

In [5]:
df.head()

,pilot,load_time
0,0,106.6
1,0,49.6
2,0,49.9
3,0,75.7
4,0,61.6


In [6]:
def get_ci_bootstrap_normal(boot_metrics: np.array, pe_metric: float, alpha: float=0.05):
    """Строит нормальный доверительный интервал.

    boot_metrics - значения метрики, полученные с помощью бутстрепа
    pe_metric - точечная оценка метрики
    alpha - уровень значимости
    
    return: (left, right) - границы доверительного интервала.
    """
    c = stats.norm.ppf(1 - alpha/2)
    se = np.std(boot_metrics)
    left, right = pe_metric - c*se, pe_metric + c*se
    return left, right

In [8]:
B = 1000
alpha = 0.5
values_a = df[df['pilot']==0]['load_time'].to_numpy()
values_b = df[df['pilot']==1]['load_time'].to_numpy()

In [13]:
quantiles = np.array([0.7, 0.74, 0.78, 0.82, 0.86, 0.9, 0.95, 0.99, 0.999, 0.9999])
for q in quantiles:
    pe = np.quantile(values_b, q) - np.quantile(values_a, q)
    bootstrap_values_a = np.random.choice(values_a, (B, len(values_a)), True)
    bootstrap_metrics_a = np.quantile(bootstrap_values_a, q, axis=1)
    bootstrap_values_b = np.random.choice(values_b, (B, len(values_b)), True)
    bootstrap_metrics_b = np.quantile(bootstrap_values_b, q, axis=1)
    bootstrap_stats = bootstrap_metrics_b - bootstrap_metrics_a
    left, right = get_ci_bootstrap_normal(bootstrap_stats, pe, alpha)
    print(f'q = {q}, ci = [{left:0.3f}, {right:0.3f}], is effect = {not(left < 0 < right)} ')

q = 0.7, ci = [0.905, 1.095], is effect = True 
q = 0.74, ci = [0.499, 0.701], is effect = True 
q = 0.78, ci = [-0.106, 0.106], is effect = False 
q = 0.82, ci = [-0.806, -0.594], is effect = True 
q = 0.86, ci = [-1.320, -1.080], is effect = True 
q = 0.9, ci = [-2.224, -1.976], is effect = True 
q = 0.95, ci = [-3.605, -3.305], is effect = True 
q = 0.99, ci = [-6.641, -5.959], is effect = True 
q = 0.999, ci = [-915.881, 137.611], is effect = False 
q = 0.9999, ci = [-115.550, 178.974], is effect = False 
